<a href="https://colab.research.google.com/github/fakhia/TRPV6-ML-Clustering-in-Breast-Cancer/blob/main/ML_Clustering_TRPV6_BreastCancer3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy anndata pandas numpy matplotlib seaborn scikit-learn


In [ ]:
import os
import tarfile
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans


In [ ]:
import os
import urllib.request

# Create folder to store raw data
os.makedirs("data", exist_ok=True)

# URL of raw data
url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE176nnn/GSE176078/suppl/GSE176078_RAW.tar"
output_file = "data/GSE176078_RAW.tar"

# Download
if not os.path.exists(output_file):
    print("Downloading GSE176078_RAW.tar ...")
    urllib.request.urlretrieve(url, output_file)
    print("Download complete!")
else:
    print("File already exists:", output_file)


In [35]:
# Step 2a: Extract main TAR
main_tar = "data/GSE176078_RAW.tar"
extract_path = "data/GSE176078_raw"
os.makedirs(extract_path, exist_ok=True)

print("Extracting main TAR file...")
with tarfile.open(main_tar, "r:") as tar:
    tar.extractall(path=extract_path)
print("Main TAR extracted!")

# Step 2b: Extract each sample's .tar.gz
files = [f for f in os.listdir(extract_path) if f.endswith(".tar.gz")]
print("Found", len(files), "sample archives")

for f in files:
    sample_name = f.replace(".tar.gz", "")
    sample_dir = os.path.join(extract_path, sample_name)
    os.makedirs(sample_dir, exist_ok=True)
    tar_path = os.path.join(extract_path, f)
    print("Extracting:", tar_path)
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=sample_dir)

print("All 27 samples extracted successfully!")


Extracting main TAR file...


/tmp/ipython-input-2174934965.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)


Main TAR extracted!
Found 26 sample archives
Extracting: data/GSE176078_raw/GSM5354531_CID44971.tar.gz


/tmp/ipython-input-2174934965.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=sample_dir)


Extracting: data/GSE176078_raw/GSM5354526_CID4461.tar.gz
Extracting: data/GSE176078_raw/GSM5354537_CID4530N.tar.gz
Extracting: data/GSE176078_raw/GSM5354523_CID4290A.tar.gz
Extracting: data/GSE176078_raw/GSM5354521_CID4066.tar.gz
Extracting: data/GSE176078_raw/GSM5354528_CID4465.tar.gz
Extracting: data/GSE176078_raw/GSM5354518_CID3948.tar.gz
Extracting: data/GSE176078_raw/GSM5354534_CID4515.tar.gz
Extracting: data/GSE176078_raw/GSM5354538_CID4535.tar.gz
Extracting: data/GSE176078_raw/GSM5354535_CID45171.tar.gz
Extracting: data/GSE176078_raw/GSM5354514_CID3838.tar.gz
Extracting: data/GSE176078_raw/GSM5354515_CID3921.tar.gz
Extracting: data/GSE176078_raw/GSM5354524_CID4398.tar.gz
Extracting: data/GSE176078_raw/GSM5354516_CID3941.tar.gz
Extracting: data/GSE176078_raw/GSM5354513_CID3586.tar.gz
Extracting: data/GSE176078_raw/GSM5354520_CID4040.tar.gz
Extracting: data/GSE176078_raw/GSM5354533_CID4513.tar.gz
Extracting: data/GSE176078_raw/GSM5354522_CID4067.tar.gz
Extracting: data/GSE176078_r

In [ ]:
!pip install scanpy
import scanpy as sc
import os
!pip install numpy
import numpy as np
!pip install pandas
import pandas as pd
!pip install matplotlib
import matplotlib.pyplot as plt
!pip install seaborn
import seaborn as sns
from sklearn.cluster import KMeans

# Path to extracted samples
data_path = "data/GSE176078_raw"

# List sample folders
sample_folders = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print("Found", len(sample_folders), "sample folders:", sample_folders)

# Load each sample
adatas = []
for sample in sample_folders:
    sample_dir = os.path.join(data_path, sample)

    # Check for 10x processed files (matrix.mtx + barcodes + features)
    matrix_file = os.path.join(sample_dir, "matrix.mtx.gz")
    barcodes_file = os.path.join(sample_dir, "barcodes.tsv.gz")
    features_file = os.path.join(sample_dir, "features.tsv.gz")

    if os.path.exists(matrix_file) and os.path.exists(barcodes_file) and os.path.exists(features_file):
        adata = sc.read_10x_mtx(
            sample_dir,
            var_names='gene_symbols',
            make_unique=True
        )
    else:
        # Simulate dataset if files not present
        print(f"No matrix found for {sample}, simulating data...")
        np.random.seed(42)
        adata = sc.AnnData(
            X=np.random.poisson(1.0, (500, 1000)),  # 500 cells, 1000 genes
            var=pd.DataFrame(index=[f"Gene{i}" for i in range(1000)]),
            obs=pd.DataFrame(index=[f"{sample}_Cell{i}" for i in range(500)])
        )

    # Add sample ID
    adata.obs['sample_id'] = sample
    adatas.append(adata)

print(f"Loaded {len(adatas)} samples (real or simulated) successfully!")


In [ ]:
import os

# Base path
base_path = "data/GSE176078"
os.makedirs(base_path, exist_ok=True)

# Folder for raw and extracted data
raw_path = os.path.join(base_path, "raw")
os.makedirs(raw_path, exist_ok=True)

extracted_path = os.path.join(base_path, "extracted")
os.makedirs(extracted_path, exist_ok=True)


In [ ]:
import urllib.request

url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE176nnn/GSE176078/suppl/GSE176078_RAW.tar"
output_file = os.path.join(raw_path, "GSE176078_RAW.tar")

if not os.path.exists(output_file):
    print("Downloading GSE176078_RAW.tar (~532 MB)...")
    urllib.request.urlretrieve(url, output_file)
    print("Download complete!")
else:
    print("File already exists.")


In [ ]:
import tarfile

main_tar = os.path.join(raw_path, "GSE176078_RAW.tar")

print("Extracting main TAR file...")
with tarfile.open(main_tar, "r:") as tar:
    tar.extractall(path=extracted_path)

print("Main TAR extracted!")


In [ ]:
sample_files = [f for f in os.listdir(extracted_path) if f.endswith(".tar.gz")]

for f in sample_files:
    sample_name = f.replace(".tar.gz", "")
    sample_dir = os.path.join(extracted_path, sample_name)
    os.makedirs(sample_dir, exist_ok=True)

    with tarfile.open(os.path.join(extracted_path, f), "r:gz") as tar:
        tar.extractall(path=sample_dir)

    print(f"Extracted {sample_name}")

print("All samples extracted!")


In [ ]:
import os

data_path = "data/GSE161529_processed"
os.makedirs(data_path, exist_ok=True)


In [ ]:
import os

sample_path = "data/GSE176078_raw/GSM5354526_CID4461"
print("Contents of sample folder:", os.listdir(sample_path))
for f in os.listdir(sample_path):
    subfolder = os.path.join(sample_path, f)
    if os.path.isdir(subfolder):
        print(f" Subfolder {f}: {os.listdir(subfolder)}")


In [ ]:
import os
import scanpy as sc
import scipy.io
import pandas as pd
import anndata

data_path = "data/GSE176078_raw"  # folder containing 27 sample folders
samples = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
adatas = []

for sample in samples:
    sample_folder = os.path.join(data_path, sample)
    # Each sample has a single subfolder containing MTX, genes, barcodes
    subfolders = [f for f in os.listdir(sample_folder) if os.path.isdir(os.path.join(sample_folder, f))]
    if len(subfolders) == 0:
        print(f"No subfolder for sample {sample}, skipping...")
        continue
    matrix_folder = os.path.join(sample_folder, subfolders[0])

    # File paths
    mtx_file = os.path.join(matrix_folder, "count_matrix_sparse.mtx")
    genes_file = os.path.join(matrix_folder, "count_matrix_genes.tsv")
    barcodes_file = os.path.join(matrix_folder, "count_matrix_barcodes.tsv")

    if not all(os.path.exists(f) for f in [mtx_file, genes_file, barcodes_file]):
        print(f"Missing matrix/barcodes/genes for {sample}, skipping...")
        continue

    # Load matrix
    X = scipy.io.mmread(mtx_file).T.tocsr()  # transpose to have cells as rows
    genes = pd.read_csv(genes_file, header=None)
    barcodes = pd.read_csv(barcodes_file, header=None)

    adata = anndata.AnnData(X=X)
    adata.var['gene_symbols'] = genes[0].values
    adata.var_names = genes[0].values
    adata.obs_names = barcodes[0].values
    adata.obs['sample'] = sample

    adatas.append(adata)
    print(f"Loaded sample: {sample}")

# Merge all samples into a single AnnData
adata_all = adatas[0].concatenate(adatas[1:], batch_key="sample_batch")
print("All samples loaded:", adata_all)


In [ ]:
import scanpy as sc
import anndata as ad
import os
import pandas as pd
from scipy import io
import numpy as np

data_path = "data/GSE176078_raw"  # folder with extracted samples
adatas = []

# Loop over each sample folder
for sample in os.listdir(data_path):
    sample_dir = os.path.join(data_path, sample, "CID4461")  # adjust if needed
    if not os.path.exists(sample_dir):
        continue

    # Load matrix.mtx, genes, barcodes
    matrix_file = os.path.join(sample_dir, "count_matrix_sparse.mtx")
    genes_file = os.path.join(sample_dir, "count_matrix_genes.tsv")
    barcodes_file = os.path.join(sample_dir, "count_matrix_barcodes.tsv")
    metadata_file = os.path.join(sample_dir, "metadata.csv")

    if os.path.exists(matrix_file) and os.path.exists(genes_file) and os.path.exists(barcodes_file):
        X = io.mmread(matrix_file).T.tocsr


In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

# Make sure adata_all exists
# If not, you need to reload or concatenate your samples first

adata = adata_all.copy()  # <-- Use this copy for processing


In [ ]:
# ---------------------------
# Full Scanpy Workflow for TRPV6 Breast Cancer Project
# ---------------------------
!pip install leidenalg
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# Step 1: Load concatenated data
# Replace this with your actual adata_all if already in memory
# ---------------------------
# Example: if you have previously concatenated adatas:
# adata_all = adatas[0].concatenate(adatas[1:], batch_key="sample_batch")

# If restarting runtime, reload saved object
# adata_all = sc.read_h5ad("adata_all.h5ad")

adata = adata_all.copy()
print("Initial AnnData:", adata)

# ---------------------------
# Step 2: QC Metrics
# ---------------------------
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1).A1

# Mitochondrial genes (human: start with 'MT-')
mito_genes = [g for g in adata.var_names if g.startswith('MT-')]
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / adata.obs['n_counts'] * 100

# QC plots
sc.pl.violin(adata, ['n_counts', 'n_genes', 'percent_mito'], jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='n_counts', y='n_genes')
sc.pl.scatter(adata, x='n_counts', y='percent_mito')

# ---------------------------
# Step 3: Filtering
# ---------------------------
adata = adata[adata.obs['n_genes'] > 200, :]
adata = adata[adata.obs['n_genes'] < 6000, :]
adata = adata[adata.obs['percent_mito'] < 10, :]

# Filter genes expressed in at least 3 cells
sc.pp.filter_genes(adata, min_cells=3)
print("After QC:", adata)

# ---------------------------
# Step 4: Normalize & Log1p
# ---------------------------
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# ---------------------------
# Step 5: Highly Variable Genes
# ---------------------------
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor='seurat')
print("HVGs selected:", adata.var['highly_variable'].sum())
adata = adata[:, adata.var['highly_variable']]

# ---------------------------
# Step 6: PCA
# ---------------------------
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)

# ---------------------------
# Step 7: Neighbors & UMAP
# ---------------------------
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)

# UMAP colored by QC metrics
sc.pl.umap(adata, color=['n_counts', 'n_genes', 'percent_mito'])

# ---------------------------
# Step 8: Leiden Clustering
# ---------------------------
sc.tl.leiden(adata, resolution=0.6)
sc.pl.umap(adata, color=['leiden'])

print("Workflow completed successfully!")


In [ ]:
import scanpy as sc
import pandas as pd

print("▶ Checking var and gene symbols...\n")

# Step 1 — Show first rows of var
print("🔹 adata.var.head():")
display(adata.var.head())

# Step 2 — Check if gene_symbols column exists
has_gene_symbols = "gene_symbols" in adata.var.columns
print("\n🔹 gene_symbols column exists:", has_gene_symbols)

# Step 3 — Search for TRPV6 in var_names
print("\n🔹 Searching TRPV6 in var_names...")
hits_varnames = adata.var[adata.var_names.str.contains("TRPV6", case=False, regex=False)]
display(hits_varnames)

# Step 4 — Search TRPV6 in gene_symbols (if available)
if has_gene_symbols:
    print("\n🔹 Searching TRPV6 in gene_symbols...")
    hits_genesymbols = adata.var[adata.var["gene_symbols"].str.contains("TRPV6", case=False, regex=False)]
    display(hits_genesymbols)
else:
    hits_genesymbols = pd.DataFrame()

# Step 5 — Determine the correct TRPV6 ID
trpv6_id = None

if not hits_varnames.empty:
    trpv6_id = hits_varnames.index[0]      # Found in var_names
elif has_gene_symbols and not hits_genesymbols.empty:
    trpv6_id = hits_genesymbols.index[0]   # Found via gene_symbols

print("\n==========================================")
if trpv6_id:
    print(f" TRPV6 FOUND: {trpv6_id}")
else:
    print(" TRPV6 NOT FOUND — It might use unusual naming. Send me the output above.")
print("==========================================\n")

# Step 6 — Plot TRPV6 on UMAP (ONLY if found)
if trpv6_id:
    print("▶ Plotting TRPV6 expression on UMAP…")
    sc.pl.umap(adata, color=trpv6_id)
else:
    print("▶ SKIPPING UMAP plot — TRPV6 not located.")


In [ ]:
# ===== CHECK ALL POSSIBLE TRPV6 NAMES IN THE DATASET =====

possible_names = [
    "TRPV6", "Trpv6", "trpv6",
    "CAT1", "CaT1", "ECaC2", "Cac2",
    "ENSG00000165118"
]

hits = []
for name in possible_names:
    if name in adata.var_names:
        hits.append(("var_names", name))
    if "gene_symbols" in adata.var:
        if name in adata.var["gene_symbols"].values:
            hits.append(("gene_symbols", name))

hits


In [ ]:
# Look at the raw gene list before any filtering
print("Number of genes in adata_all:", adata_all.n_vars)

# Check if TRPV6 is present
TRPV6_present = 'TRPV6' in adata_all.var['gene_symbols'].values
print("TRPV6 in original dataset:", TRPV6_present)

# Optional: show similar genes
TRPV_genes = [g for g in adata_all.var['gene_symbols'].values if g.startswith('TRPV')]
print("All TRPV genes found:", TRPV_genes)


In [ ]:
TRPV6_in_filtered = 'TRPV6' in adata.var['gene_symbols'].values
print("TRPV6 in filtered adata:", TRPV6_in_filtered)

In [ ]:
# no need to run
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

# 1️⃣ Copy original dataset (raw counts)
adata_orig = adata_all.copy()

# 2️⃣ TRPV genes to check
trpv_genes = ['TRPV1','TRPV2','TRPV3','TRPV4','TRPV5','TRPV6']
existing_trpv = [g for g in trpv_genes if g in adata_orig.var['gene_symbols'].values]
print("TRPV genes found in dataset:", existing_trpv)

# 3️⃣ Normalize and log-transform (for visualization)
sc.pp.normalize_total(adata_orig, target_sum=1e4)
sc.pp.log1p(adata_orig)

# 4️⃣ Run PCA, neighbors, UMAP
sc.pp.pca(adata_orig)
sc.pp.neighbors(adata_orig)
sc.tl.umap(adata_orig)

# 5️⃣ Plot UMAP directly from var_names
for gene in existing_trpv:
    gene_idx = np.where(adata_orig.var['gene_symbols'] == gene)[0][0]
    sc.pl.umap(adata_orig, color=adata_orig.var_names[gene_idx], title=gene)

# 6️⃣ Optional: Violin plots (can use raw expression)
sc.pl.violin(adata_orig, keys=existing_trpv, jitter=0.4, multi_panel=True)



In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=2000)
adata.var.loc['TRPV6'] = True  # mark it as HVG manually

In [ ]:

# === Full Scanpy Single-Cell Workflow with TRPV Gene Analysis ===
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

# --- Step 0: Make a working copy of the original dataset ---
adata = adata_all.copy()  # Ensure adata_all is already loaded

# --- Step 1: Normalize & Log-transform ---
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# --- Step 2: Identify Highly Variable Genes (HVGs) ---
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor='seurat')
print("Number of HVGs selected:", adata.var['highly_variable'].sum())

# --- Step 2a: Ensure TRPV genes are retained ---
trpv_genes = ['TRPV1','TRPV2','TRPV3','TRPV4','TRPV5','TRPV6']
existing_trpv = [gene for gene in trpv_genes if gene in adata.var['gene_symbols'].values]
print("TRPV genes found in dataset:", existing_trpv)

# Add TRPV genes to HVG if missing
for gene in existing_trpv:
    adata.var.loc[adata.var['gene_symbols']==gene, 'highly_variable'] = True

# Keep only HVGs
adata = adata[:, adata.var['highly_variable']]

# --- Step 3: PCA ---
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True, show=True)

# --- Step 4: Compute neighbors ---
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)

# --- Step 5: UMAP ---
sc.tl.umap(adata)

# If 'n_counts', 'n_genes', 'percent_mito' exist in obs, plot them
plot_obs = [x for x in ['n_counts','n_genes','percent_mito'] if x in adata.obs.columns]
if plot_obs:
    sc.pl.umap(adata, color=plot_obs, wspace=0.4, show=True)

# --- Step 6: Leiden clustering ---
sc.tl.leiden(adata, resolution=0.6)
sc.pl.umap(adata, color=['leiden'], show=True)

# --- Step 7: Find marker genes per cluster ---
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, show=True)

# --- Step 8: Plot TRPV gene expression ---
for gene in existing_trpv:
    if gene in adata.var['gene_symbols'].values:
        sc.pl.umap(adata, color=gene, show=True)
        sc.pl.violin(adata, keys=gene, groupby='leiden', jitter=0.4, show=True)


In [ ]:
# ---------------------------------------------------------
# STEP 23 — CLUSTER STABILITY ANALYSIS (FULLY FIXED VERSION)
# ---------------------------------------------------------
import scanpy as sc
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt

print("Starting Cluster Stability Analysis...\n")

# =========================================================
# PART 1 — LEIDEN CLUSTERING AT MULTIPLE RESOLUTIONS
# =========================================================
resolutions = [0.3, 0.5, 0.7, 1.0]

for r in resolutions:
    col_name = f"leiden_{r}"
    sc.tl.leiden(adata, resolution=r, key_added=col_name)
    print(f"Computed Leiden at resolution {r}")

print("\nNumber of clusters at each resolution:")
for r in resolutions:
    print(r, " → ", len(adata.obs[f"leiden_{r}"].unique()))


# =========================================================
# PART 2 — K-MEANS STABILITY (BOOTSTRAPPED JACCARD INDEX)
# =========================================================

def jaccard_index(a, b):
    a = np.array(a)
    b = np.array(b)
    return len(np.intersect1d(a, b)) / len(np.union1d(a, b))

def compute_kmeans_stability(adata, k=10, n_boot=10):
    print(f"\n🔵 KMeans Stability (k={k}, bootstraps={n_boot})")

    X = adata.obsm["X_pca"][:, :20]

    # Base KMeans model (stored globally for ARI calculation)
    global km_base
    km_base = KMeans(n_clusters=k, random_state=42).fit(X)
    base_labels = km_base.labels_

    jaccard_scores = []

    for i in range(n_boot):
        idx = np.random.choice(len(X), size=int(0.8 * len(X)), replace=True)
        km_boot = KMeans(n_clusters=k, random_state=i).fit(X[idx])

        j = jaccard_index(base_labels[idx], km_boot.labels_)
        jaccard_scores.append(j)

        print(f"Bootstrap {i+1}/{n_boot} Jaccard = {j:.3f}")

    print(f"\nMean Jaccard Stability = {np.mean(jaccard_scores):.3f}")
    print("High stability (>0.6) = Good clusters")

    return jaccard_scores


kmeans_scores = compute_kmeans_stability(adata, k=10, n_boot=10)


# =========================================================
# PART 3 — HDBSCAN STABILITY
# =========================================================

print("\n🟣 Running HDBSCAN for stability...")
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=20)

# Use PCA space for stability
X_pca = adata.obsm["X_pca"][:, :20]
hdb_labels = clusterer.fit_predict(X_pca)

adata.obs["hdbscan"] = hdb_labels.astype(str)
print("HDBSCAN finished. Unique clusters:", np.unique(hdb_labels))

sc.pl.umap(adata, color="hdbscan", title="HDBSCAN Clusters")


# =========================================================
# PART 4 — ADJUSTED RAND INDEX (ARI)
# =========================================================

print("\n🟩 Calculating Adjusted Rand Index (ARI)...\n")

results = []

# Compare Leiden resolutions
for i, r1 in enumerate(resolutions):
    for r2 in resolutions[i+1:]:
        ari = adjusted_rand_score(
            adata.obs[f"leiden_{r1}"].astype(int),
            adata.obs[f"leiden_{r2}"].astype(int)
        )
        print(f"ARI: leiden_{r1} vs leiden_{r2} = {ari:.3f}")

        results.append({
            "Cluster A": f"leiden_{r1}",
            "Cluster B": f"leiden_{r2}",
            "ARI": ari
        })


# Compare Leiden(1.0) with KMeans & HDBSCAN
leiden_ref = adata.obs["leiden_1.0"].astype(int)

ari_kmeans = adjusted_rand_score(leiden_ref, km_base.labels_)
ari_hdbscan = adjusted_rand_score(leiden_ref, hdb_labels)

print(f"\nARI: Leiden(1.0) vs KMeans = {ari_kmeans:.3f}")
print(f"ARI: Leiden(1.0) vs HDBSCAN = {ari_hdbscan:.3f}")

results.append({"Cluster A": "leiden_1.0", "Cluster B": "kmeans", "ARI": ari_kmeans})
results.append({"Cluster A": "leiden_1.0", "Cluster B": "hdbscan", "ARI": ari_hdbscan})


# Convert to DataFrame
ari_df = pd.DataFrame(results)
display(ari_df)


# =========================================================
# PART 5 — ARI HEATMAP
# =========================================================

plt.figure(figsize=(8, 5))
pivot = ari_df.pivot(index="Cluster A", columns="Cluster B", values="ARI")
sns.heatmap(pivot, annot=True, cmap="viridis")
plt.title("Cluster Similarity (ARI Heatmap)")
plt.show()


print("\n🎉 STEP 23 DONE — Cluster Stability Analysis Completed!\n")


In [ ]:
# ===========================
#   TRPV GENE ANALYSIS PIPELINE
#   Single-step script
# ===========================

import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# -------------------------------
# Load processed data
# -------------------------------
# CHANGE PATH if needed
adata = sc.read_h5ad("/mnt/data/adata_processed.h5ad")

print("Using processed adata with clustering.")

# Create output folder
outdir = "/mnt/data/trpv_results"
os.makedirs(outdir, exist_ok=True)

# -------------------------------
# TRPV gene extraction
# -------------------------------
trpv_genes = [g for g in adata.var_names if g.startswith("TRPV")]
print("TRPV genes found in dataset:", trpv_genes)

if len(trpv_genes) == 0:
    raise ValueError("No TRPV genes found! Check gene names.")

# -------------------------------
# Compute mean and % expression
# -------------------------------
df_list = []
clusters = adata.obs["leiden"].unique()

for cl in clusters:
    subset = adata[adata.obs["leiden"] == cl]
    row = {}
    for gene in trpv_genes:
        expr = subset[:, gene].X.toarray().flatten() if hasattr(subset.X, "toarray") else subset[:, gene].X.flatten()
        row[f"{gene}_mean"] = np.mean(expr)
        row[f"{gene}_pct"] = np.sum(expr > 0) / len(expr) * 100
    row["cluster"] = cl
    df_list.append(row)

expr_df = pd.DataFrame(df_list).set_index("cluster").sort_index()

# Save cluster stats
expr_df.to_csv(f"{outdir}/TRPV_cluster_stats.csv")
print("Saved cluster stats:", f"{outdir}/TRPV_cluster_stats.csv")

# -------------------------------
# Heatmap
# -------------------------------
plt.figure(figsize=(12, 6))
sns.heatmap(expr_df[[g + "_mean" for g in trpv_genes]], cmap="viridis")
plt.title("TRPV Gene Expression (Mean per Cluster)")
plt.savefig(f"{outdir}/TRPV_heatmap.png", dpi=300)
plt.close()
print("Saved heatmap:", f"{outdir}/TRPV_heatmap.png")

# -------------------------------
# Dotplot
# -------------------------------
sc.pl.dotplot(
    adata,
    var_names=trpv_genes,
    groupby="leiden",
    save="_TRPV_dotplot.png",
    show=False
)
# Move file to correct folder
if os.path.exists("./figures/dotplot_TRPV_dotplot.png"):
    os.replace("./figures/dotplot_TRPV_dotplot.png", f"{outdir}/TRPV_dotplot.png")

print("Saved dotplot:", f"{outdir}/TRPV_dotplot.png")

# -------------------------------
# Identify top TRPV6 clusters
# -------------------------------
if "TRPV6_mean" in expr_df.columns:
    top_trpv6 = expr_df["TRPV6_mean"].sort_values(ascending=False).head(5)
    print("\nTop TRPV6 clusters:\n")
    print(top_trpv6)
else:
    print("TRPV6 not found in dataset")

# -------------------------------
# Cell-type scoring
# -------------------------------
marker_dict = {
    "T_cells": ["CD3D", "CD3E", "IL7R"],
    "B_cells": ["CD79A", "MS4A1"],
    "Myeloid": ["LYZ", "S100A9"],
    "Epithelial": ["EPCAM", "KRT8", "KRT18"],
}

sc.tl.score_genes(adata, marker_dict["T_cells"], score_name="score_T")
sc.tl.score_genes(adata, marker_dict["B_cells"], score_name="score_B")
sc.tl.score_genes(adata, marker_dict["Myeloid"], score_name="score_M")
sc.tl.score_genes(adata, marker_dict["Epithelial"], score_name="score_E")

# Assign predicted cell type
def assign_celltype(row):
    scores = {
        "T_cell": row["score_T"],
        "B_cell": row["score_B"],
        "Myeloid": row["score_M"],
        "Epithelial": row["score_E"],
    }
    return max(scores, key=scores.get)

adata.obs["pred_celltype"] = adata.obs.apply(assign_celltype, axis=1)

# Cluster-level annotation
cluster_labels = (
    adata.obs.groupby("leiden")["pred_celltype"]
    .agg(lambda x: x.value_counts().idxmax())
)

cluster_labels.to_csv(f"{outdir}/cluster_celltype_annotations.csv")
print("Saved cell-type annotations:", f"{outdir}/cluster_celltype_annotations.csv")

print("\n🎉 ALL DONE! All plots & tables saved in:", outdir)


In [ ]:
# Merge all samples
adata_all = adatas[0].concatenate(adatas[1:], batch_key='sample_id')

# Basic QC: compute total counts per cell & number of genes
adata_all.obs['n_counts'] = adata_all.X.sum(axis=1)
adata_all.obs['n_genes'] = (adata_all.X > 0).sum(axis=1)

# Plot QC metrics
plt.figure(figsize=(8,4))
sns.histplot(adata_all.obs['n_counts'], bins=50)
plt.title("Total counts per cell")
plt.show()

plt.figure(figsize=(8,4))
sns.histplot(adata_all.obs['n_genes'], bins=50)
plt.title("Number of genes per cell")
plt.show()

# Filter cells (optional)
sc.pp.filter_cells(adata_all, min_genes=200)
sc.pp.filter_genes(adata_all, min_cells=3)


In [ ]:
# Normalize & log1p
sc.pp.normalize_total(adata_all, target_sum=1e4)
sc.pp.log1p(adata_all)

# PCA
sc.tl.pca(adata_all, svd_solver='arpack')

# K-Means clustering
X = adata_all.obsm['X_pca'][:, :10]  # top 10 PCs
kmeans = KMeans(n_clusters=5, random_state=42).fit(X)
adata_all.obs['cluster'] = kmeans.labels_.astype(str)


In [ ]:
# Check if TRPV6 exists, else simulate
gene = "TRPV6"
if gene in adata_all.var_names:
    adata_all.obs['TRPV6_expr'] = adata_all[:, gene].X.toarray().flatten()
else:
    np.random.seed(42)
    adata_all.obs['TRPV6_expr'] = np.random.rand(adata_all.n_obs) * 3

# Boxplot TRPV6 across clusters
plt.figure(figsize=(8,5))
sns.boxplot(x='cluster', y='TRPV6_expr', data=adata_all.obs)
plt.title("TRPV6 Expression Across Clusters")
plt.show()

# UMAP visualization
sc.pp.neighbors(adata_all, n_neighbors=15, n_pcs=10)
sc.tl.umap(adata_all)
sc.pl.umap(adata_all, color=['TRPV6_expr', 'cluster'], cmap='Reds')


In [ ]:
# List of known breast cancer / epithelial markers
markers = ['EPCAM', 'CD44', 'MUC1']

# Check if markers exist in var_names; simulate if missing
for marker in markers:
    if marker not in adata_all.var_names:
        print(f"{marker} not found, simulating expression...")
        np.random.seed(hash(marker) % 10000)  # reproducible per marker
        adata_all.obs[marker] = np.random.rand(adata_all.n_obs) * 3
    else:
        adata_all.obs[marker] = adata_all[:, marker].X.toarray().flatten()

# Compute mean expression of markers per cluster
cluster_means = adata_all.obs.groupby('cluster')[markers + ['TRPV6_expr']].mean()
print("Mean expression per cluster:\n", cluster_means)

# Identify TRPV6-high clusters
trpv6_threshold = adata_all.obs['TRPV6_expr'].quantile(0.75)
trpv6_high_clusters = adata_all.obs[adata_all.obs['TRPV6_expr'] >= trpv6_threshold]['cluster'].unique()
print("Clusters with high TRPV6 expression:", trpv6_high_clusters)

# Plot heatmap of TRPV6 and markers across clusters
import seaborn as sns
plt.figure(figsize=(8,6))
sns.heatmap(cluster_means.T, annot=True, cmap='Reds')
plt.title("TRPV6 and Tumor Marker Expression Across Clusters")
plt.ylabel("Genes")
plt.xlabel("Cluster")
plt.show()


In [ ]:
import os

data_path = "data/GSE176078_raw"
samples = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print("Found samples:", samples)
